# Tutorial-Fast mode data generation

This tutorial explains the steps to be followed to generate the data that will be used in the fast
calculation mode. In this example, the fast mode data for"Air_30kW" model is generated.

In [ ]:
import json
from mosaik_heatpump.heatpump.Heat_Pump_Design import Heat_Pump_Design
import pandas as pd
import numpy as np
from tqdm import tqdm
from tespy.tools import logger
import logging
logger.define_logging(log_path=True, log_version=True,
                      screen_level=logging.ERROR,
                      file_level=logging.DEBUG)

In [ ]:
# Enter the name of the new heat pump model. Should match the name used in
# the eta_s_data.json
model_name = 'Air_30kW'

In [ ]:
params_air = {
        'calc_mode': 'detailed',
        'hp_model': model_name,
        'heat_source': 'Air',
    }

heat_pump_1 = Heat_Pump_Design(params_air)

In [ ]:
with open("../../mosaik_heatpump/heatpump/eta_s_data.json") as feedsjson:
    data_1 = json.load(feedsjson)

etas_dict = data_1[heat_pump_1.hp_model]['eta_s']
heatload_dict = data_1[heat_pump_1.hp_model]['heatload']

In [ ]:
# The minimum and maximum heat source temperatures
heat_source_T_min = min(list(map(int, etas_dict)))
heat_source_T_max = max(list(map(int, etas_dict)))

# The minimum and maximum heating capacity of the heat pump
hl_min = data_1[heat_pump_1.hp_model]['min_heatload']
hl_max = data_1[heat_pump_1.hp_model]['max_heatload']

# The minimum and maximum condenser water outlet temperatures
cons_T_min = min(list(map(int, etas_dict[str(heat_source_T_max)])))
cons_T_max = max(list(map(int, etas_dict[str(heat_source_T_max)])))

# The lists for the whole range of operation of the heat pump
heat_source_T_list = np.arange(heat_source_T_min, heat_source_T_max + 1, 1)
cond_in_T_list = np.arange(cons_T_min-5, cons_T_max-4, 1)

df = pd.DataFrame(index=heat_source_T_list, columns=cond_in_T_list)

In [ ]:
pbar = tqdm(total=len(heat_source_T_list))

for Temp in heat_source_T_list:
    pbar1 = tqdm(total=len(cond_in_T_list))
    for cond_in_T in cond_in_T_list:
        skip = False
        heat_pump_1.heat_source_T = Temp
        heat_pump_1.cond_in_T = cond_in_T
        heat_pump_1._etas_heatload_id()

        if not heat_pump_1.skip_step:

            try:
                heat_pump_1._design_hp()
            except:
                print('Error for heat_source_T : {} & Cond_in_T : {}'.format(Temp, cond_in_T))
                skip = True

            if (hl_min +500) < heat_pump_1.heatload_des:
                heatload_list = np.arange(hl_min, heat_pump_1.heatload_des, 500)
                if heatload_list[-1] < heat_pump_1.heatload_des:
                    heatload_list = np.append(heatload_list, heat_pump_1.heatload_des)
            else:
                heatload_list = [heat_pump_1.heatload_des]

            save_dict = {}

            if skip is False:
                for heatload in heatload_list:
                    inputs = {'heat_source_T': Temp, 'cond_in_T': cond_in_T, 'Q_Demand': heatload}

                    try:
                        heat_pump_1.step(inputs)
                        save_dict[heatload] = {'cond_m': heat_pump_1.cond_m, 'COP': heat_pump_1.COP}
                        if heat_pump_1.cond_m == 0 and heat_pump_1.COP == 0:
                            print("Error for {} inputs".format(inputs))
                    except:
                        print("Error for {} inputs".format(inputs))
                        save_dict[heatload] = {'cond_m': 0, 'COP': 0}
            else:
                # print("Error for {} inputs".format(inputs))
                for heatload in heatload_list:
                    save_dict[heatload] = {'cond_m': 0, 'COP': 0}
        else:
            print('Skipping heat_source_T : {} & Cond_in_T : {}'.format(Temp, cond_in_T))
            save_dict = {}

        df.at[Temp, cond_in_T] = save_dict
        pbar1.update()
    pbar1.close()
    pbar.update()

pbar.close()

In [ ]:
result = df.to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed, indent=4)

In [ ]:
with open("../../mosaik_heatpump/heatpump/COP_m_data.json") as feedsjson:
    feeds = json.load(feedsjson)

if model_name in feeds.keys():
    a = input('The model name already exists and must be replaced. Do you want to replace the data (y/n)? : ')
    if a.lower() == 'y':
        feeds[model_name]= parsed
    elif a.lower() =='n':
        model_name = input('Enter new model name')
        feeds[model_name]= parsed
    else:
        print("Wrong input. Execute cell again.")
else:
    feeds[model_name] = parsed

In [ ]:
with open("../../mosaik_heatpump/heatpump/COP_m_data.json", "w") as write_file:
    json.dump(feeds, write_file, indent=4)